#### Implementaion of Generative Adversarial Network

In [1]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

In [2]:
from utils import Logger

In [39]:
def mnist_data():
    compose = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ])
    out_dir = "./dataset"
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)
    
data = mnist_data()
data_loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=True)
num_of_batches = len(data_loader)

In [4]:
class DiscriminatorNet(nn.Module):
    
    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        n_features = 784
        n_out = 1
        
        self.hidden0 = nn.Sequential(
        nn.Linear(n_features, 1024),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.3)
        )
        self.hidden1 = nn.Sequential(
        nn.Linear(1024, 512),
        nn.LeakyReLU(0.2), 
        nn.Dropout(0.3)
        )
        self.hidden2 = nn.Sequential(
        nn.Linear(512, 256),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
        nn.Linear(256, n_out),
        nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x
    
discriminator = DiscriminatorNet()

In [5]:
def images_to_vectors(images):
    return images.view(images.size(0), 784)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 28, 28)

In [6]:
class GeneratorNet(nn.Module):
    
    def __init__(self):
        super(GeneratorNet, self).__init__()
        n_features = 100
        n_out = 784
        
        self.hidden0 = nn.Sequential(
        nn.Linear(n_features, 256),
        nn.LeakyReLU(0.2)
        )
        self.hidden1 = nn.Sequential(
        nn.Linear(256, 512),
        nn.LeakyReLU(0.2)
        )
        self.hidden2 = nn.Sequential(
        nn.Linear(512, 1024),
        nn.LeakyReLU(0.2)
        )
        self.out = nn.Sequential(
        nn.Linear(1024, n_out),
        nn.Tanh()
        )
        
    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x

generator = GeneratorNet()

In [7]:
def noise(size):
    n = Variable(torch.rand(size, 100))
    return n

In [8]:
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

In [9]:
loss = nn.BCELoss

In [10]:
def ones_target(size):
    data = Variable(torch.ones(size, 1))
    return data

def zeros_target(size):
    data = Variable(torch.zeros(size, 1))
    return data

In [41]:
def train_discriminator(optimizer, real_data, fake_data):
    N = real_data.size(0)
    optimizer.zero_grad()
    
    prediction_real = discriminator(real_data)
    error_real = loss(prediction_real, ones_target(N))
    error_real.backward()
    
    prediction_fake = discriminator(fake_data)
    error_fake = loss(prediction_fake, zeros_target(N))
    error_fake.backward()
    
    optimizer.step()
    
    return error_real + error_fake, prediction_real, prediction_fake

In [12]:
def train_generator(optimizer, fake_data):
    N = fake_data.size(0)
    optimizer.zero_grad()
    prediction = discriminator(fake_data)
    error = loss(prediction, ones_target(N))
    error.backward()
    
    optimizer.step()
    
    return error

In [14]:
num_test_samples = 16
test_noise = noise(num_test_samples)

In [42]:
logger = Logger(model_name="VGAN", data_name="MNIST")
num_epochs = 200

for epoch in range(num_epochs):
    for n_batch, (real_batch, _) in enumerate(data_loader):
        N = real_batch.size(0)
        
        real_data = Variable(images_to_vectors(real_batch))
        fake_data = generator(noise(N)).detach()
        
        d_error, d_pred_real, d_pred_fake = train_discriminator(optimizer=d_optimizer, real_data=real_data, fake_data=fake_data)
        
        fake_data = generator(noise(N))
        g_error = train_generator(optimizer=g_optimizer, fake_data=fake_data)
        
        logger.log(d_error, g_error, epoch, n_batch, num_of_batches)
        
        if n_batch % 100 == 0:
            test_images = vectors_to_images(generator(test_noise))
            test_images = test_images.data
            
            logger.log_images(
            test_images, num_test_samples,
            epoch, n_batch, num_of_batches
            );
            logger.display_status(
            epoch, num_epochs, n_batch, num_of_batches,
            d_error, g_error, d_pred_real, d_pred_fake
            )

RuntimeError: bool value of Tensor with more than one value is ambiguous

In [38]:
data_loader.dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./dataset
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=0.5, std=0.5)
           )

TypeError: 'BatchSampler' object is not subscriptable